In [1]:
from __future__ import division
import os
import logging
import time
import glob
import datetime
import argparse
import numpy as np
from scipy.io import loadmat, savemat

import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from arch_unet import UNet
import utils as util
from collections import OrderedDict
from tqdm import tqdm

from Sber_utils import Masker, save_network, load_network, save_state, resume_state, checkpoint, generate_mask, interpolate_mask, depth_to_space, get_generator

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--noisetype", type=str, default="gauss25", choices=['gauss25', 'gauss5_50', 'poisson30', 'poisson5_50'])
parser.add_argument('--resume', type=str)
parser.add_argument('--checkpoint', type=str)
parser.add_argument('--train_dir', type=str, default='./dataset/small/train')
parser.add_argument('--test_dir', type=str, default='./dataset/small/validation')
parser.add_argument('--save_test_path', type=str, default='./test')
parser.add_argument('--save_model_path', type=str, default='../experiments/small')
parser.add_argument('--log_name', type=str, default='xxx_b2u_unet_fmdd_112rf20')
parser.add_argument('--gpu_devices', default='0', type=str)
parser.add_argument('--parallel', action='store_true')
parser.add_argument('--n_feature', type=int, default=48)
parser.add_argument('--n_channel', type=int, default=1)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--w_decay', type=float, default=1e-8)
parser.add_argument('--gamma', type=float, default=0.5)
parser.add_argument('--n_epoch', type=int, default=100)
parser.add_argument('--n_snapshot', type=int, default=1)
parser.add_argument('--batchsize', type=int, default=4)
parser.add_argument('--patchsize', type=int, default=128)
parser.add_argument("--Lambda1", type=float, default=1.0)
parser.add_argument("--Lambda2", type=float, default=2.0)
parser.add_argument("--increase_ratio", type=float, default=20.0)
parser.add_argument('--test_dataset_name', type=str)
parser.add_argument('--train_dataset_name', type=str)
parser.add_argument('--repeat_times', type=int, default=3)



opt = parser.parse_args([
                        '--train_dir', './data/train', 
                        '--test_dir', './data/test',
                        '--save_test_path', './test',
                        '--save_model_path', './experiments/my_models', 
                        '--log_name', 'b2u_second',
                        '--test_dataset_name', 'small_b2u_g209',
                        '--train_dataset_name', 'b2u_g209',
                        '--Lambda1', '1.0',
                        '--Lambda2', '2.0',
                        '--increase_ratio', '20.0',
                        '--n_epoch', '100',
                        '--batchsize', '2'
                        ])
                        
systime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
operation_seed_counter = 0
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpu_devices
torch.set_num_threads(8)

opt.save_path = os.path.join(opt.save_model_path, opt.log_name)
os.makedirs(opt.save_path, exist_ok=True)

In [3]:
def apply_median(img, percentile):
    tresh = np.percentile(img.ravel(), percentile)
    mask = np.zeros_like(img)
    mask[img < tresh] = 1
    median_img = cv2.medianBlur(img, 7)
    img = img*(1 - mask) + median_img * mask
    return img

In [4]:
class TestMicroscopyDataset(Dataset):
    def __init__(self, img_dir,channels=3, transform=None, apply_median=False):
        self.img_dir = img_dir
        self.apply_median = apply_median
        self.transform = transform
        self.img_filenames = os.listdir(img_dir)
        if channels == 1:
            self.channels = 0
        else:
            self.channels = channels

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        name = os.path.split(self.img_filenames[idx])[-1].split('.')[0]

        image = cv2.imread(os.path.join(self.img_dir, self.img_filenames[idx]), self.channels)


        if self.apply_median:
            image = apply_median(image, 10)

        if self.transform:
            image = self.transform(image)

        return image, name

In [5]:
class TrainMicroscopyDataset(Dataset):
    def __init__(self, img_dir,channels=3, transform=None, patch=256, apply_median=False):
        self.img_dir = img_dir
        self.apply_median = apply_median
        self.transform = transform
        self.img_filenames = os.listdir(img_dir)
        if channels == 1:
            self.channels = 0
        else:
            self.channels = channels

        self.patch = patch
        print('fetch {} samples for training'.format(len(self.img_filenames)))

    def __getitem__(self, idx):
        # fetch image
        name = os.path.split(self.img_filenames[idx])[-1].split('.')[0]

        image = cv2.imread(os.path.join(self.img_dir, self.img_filenames[idx]), self.channels)
        # random crop
        if len(image.shape) == 2:
            H = image.shape[0]
            W = image.shape[1]

        CSize = self.patch
        rnd_h = np.random.randint(0, max(0, H - CSize))
        rnd_w = np.random.randint(0, max(0, W - CSize))
        image = image[rnd_h : rnd_h + CSize, rnd_w : rnd_w + CSize]


        if self.apply_median:
            image = apply_median(image, 10)

        if self.transform:
            image = self.transform(image)

        return image, name

    def __len__(self):
        return len(self.img_filenames)

In [6]:
# Dataset
train_transforms = transforms.Compose([transforms.ToTensor()])
train_dataset_dir = os.path.join(opt.train_dir, opt.train_dataset_name)
train_dataset = TrainMicroscopyDataset(train_dataset_dir,opt.n_channel, transform=train_transforms, apply_median=True)
train_dataloader = DataLoader(dataset=train_dataset, num_workers=8, batch_size=opt.batchsize, shuffle=True)

test_transforms = transforms.Compose([transforms.ToTensor()])
test_dataset_dir = os.path.join(opt.test_dir, opt.test_dataset_name)
test_dataset = TestMicroscopyDataset(test_dataset_dir,opt.n_channel, transform=test_transforms, apply_median=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# Masker
masker = Masker(width=4, mode='interpolate', mask_type='all')
# Network
network = UNet(in_channels=opt.n_channel,
                out_channels=opt.n_channel,
                wf=opt.n_feature)
if opt.parallel:
    network = torch.nn.DataParallel(network)
network = network.cuda()

# validation
save_test_path = os.path.join(opt.save_test_path, opt.test_dataset_name)
os.makedirs(save_test_path, exist_ok=True)

validation_path = os.path.join(save_test_path, opt.log_name)
os.makedirs(validation_path, exist_ok=True)
# np.random.seed(101)

# about training scheme
num_epoch = opt.n_epoch
ratio = num_epoch / 100
optimizer = optim.Adam(network.parameters(), lr=opt.lr,
                       weight_decay=opt.w_decay)
scheduler = lr_scheduler.MultiStepLR(optimizer,
                                     milestones=[
                                         int(20 * ratio) - 1,
                                         int(40 * ratio) - 1,
                                         int(60 * ratio) - 1,
                                         int(80 * ratio) - 1
                                     ],
                                     gamma=opt.gamma)
print("Batchsize={}, number of epoch={}".format(opt.batchsize, opt.n_epoch))

# Resume and load pre-trained model
epoch_init = 1
if opt.resume is not None:
    epoch_init, optimizer, scheduler = resume_state(opt.resume, optimizer, scheduler)
if opt.checkpoint is not None:
    network = load_network(opt.checkpoint, network, strict=True)

# temp
if opt.checkpoint is not None:
    epoch_init = 65
    for i in range(1, epoch_init):
        scheduler.step()
        new_lr = scheduler.get_lr()[0]
        logger.info('----------------------------------------------------')
        logger.info("==> Resuming Training with learning rate:{}".format(new_lr))
        logger.info('----------------------------------------------------')

print('init finish')


Thread1 = 0.4
Thread2 = 1.0
Lambda1 = opt.Lambda1
Lambda2 = opt.Lambda2
increase_ratio = opt.increase_ratio


fetch 457 samples for training
Batchsize=2, number of epoch=100
init finish


In [7]:
for epoch in range(epoch_init, opt.n_epoch + 1):
    cnt = 0

    for param_group in optimizer.param_groups:
        current_lr = param_group['lr']
    print("LearningRate of Epoch {} = {}".format(epoch, current_lr))

    network.train()
    for noisy, name in tqdm(train_dataloader):
        st = time.time()
        # noisy = noisy / 255.0
        # print(noisy.shape)

        noisy = noisy.cuda()
        # print('noisy_mean', noisy.mean())

        optimizer.zero_grad()

        net_input, mask = masker.train(noisy)
        noisy_output = network(net_input)
        n, c, h, w = noisy.shape
        noisy_output = (noisy_output*mask).view(n, -1, c, h, w).sum(dim=1)
        diff = noisy_output - noisy

        with torch.no_grad():
            exp_output = network(noisy)
        exp_diff = exp_output - noisy

        # g25, p30: 1_1-2; frange-10
        # g5-50 | p5-50 | raw; 1_1-2; range-10
        Lambda = epoch / opt.n_epoch
        if Lambda <= Thread1:
            beta = Lambda2
        elif Thread1 <= Lambda <= Thread2:
            beta = Lambda2 + (Lambda - Thread1) * \
                (increase_ratio-Lambda2) / (Thread2-Thread1)
        else:
            beta = increase_ratio
        alpha = Lambda1

        revisible = diff + beta * exp_diff
        loss_reg = alpha * torch.mean(diff**2)
        loss_rev = torch.mean(revisible**2)
        loss_all = loss_reg + loss_rev

        loss_all.backward()
        optimizer.step()
        
    scheduler.step()

    if epoch % opt.n_snapshot == 0 or epoch == opt.n_epoch:
        network.eval()
        # save checkpoint
        save_network(network, epoch, "model", opt=opt)
        save_state(epoch, optimizer, scheduler, opt=opt)

        # validation
        np.random.seed(101)

        save_dir = os.path.join(validation_path, f'epoch_{epoch}')
        os.makedirs(save_dir, exist_ok=True)


        # for idx in range(num_img):
        for im, name in test_dataloader:
            
            if type(im) == torch.Tensor:
                if len(im.shape) == 4: 
                    im = im.squeeze(0)
                im = im.permute(1,2,0)
                im = im.numpy()
            origin255 = im.copy() * 255
            origin255 = origin255.astype(np.uint8)
            im = np.array(im, dtype=np.float32) #/ 255.0
            noisy_im = im.copy() # add
            noisy255 = noisy_im.copy()
            noisy255 = np.clip(noisy255 * 255.0 + 0.5, 0,
                                255).astype(np.uint8)

            # padding to square
            H = noisy_im.shape[0]
            W = noisy_im.shape[1]
            val_size = (max(H, W) + 31) // 32 * 32
            noisy_im = np.pad(
                noisy_im,
                [[0, val_size - H], [0, val_size - W], [0, 0]],
                'reflect')
            transformer = transforms.Compose([transforms.ToTensor()])
            noisy_im = transformer(noisy_im)
            noisy_im = torch.unsqueeze(noisy_im, 0)
            noisy_im = noisy_im.cuda()
            with torch.no_grad():
                n, c, h, w = noisy_im.shape
                net_input, mask = masker.train(noisy_im)
                noisy_output = (network(net_input) *
                                mask).view(n, -1, c, h, w).sum(dim=1)
                dn_output = noisy_output.detach().clone()
                # Release gpu memory
                del net_input, mask, noisy_output
                torch.cuda.empty_cache()
                exp_output = network(noisy_im)
            pred_dn = dn_output[:, :, :H, :W]
            pred_exp = exp_output.detach().clone()[:, :, :H, :W]
            # print('pred_exp mean', pred_exp.mean())
            pred_mid = (pred_dn + beta*pred_exp) / (1 + beta)

            # Release gpu memory
            del exp_output
            torch.cuda.empty_cache()

            pred_dn = pred_dn.permute(0, 2, 3, 1)
            pred_exp = pred_exp.permute(0, 2, 3, 1)
            pred_mid = pred_mid.permute(0, 2, 3, 1)

            pred_dn = pred_dn.cpu().data.clamp(0, 1).numpy().squeeze(0)
            pred_exp = pred_exp.cpu().data.clamp(0, 1).numpy().squeeze(0)
            pred_mid = pred_mid.cpu().data.clamp(0, 1).numpy().squeeze(0)

            pred255_dn = np.clip(pred_dn * 255.0 + 0.5, 0,
                                    255).astype(np.uint8)
            pred255_exp = np.clip(pred_exp * 255.0 + 0.5, 0,
                                    255).astype(np.uint8)
            pred255_mid = np.clip(pred_mid * 255.0 + 0.5, 0,
                                    255).astype(np.uint8)


            # visualization
            if opt.n_channel == 1:
                color_mode = 'L'
                save_path = os.path.join(save_dir, f"{name}_clean.png")
                Image.fromarray(origin255.squeeze()).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_noisy.png")    
                Image.fromarray(noisy255.squeeze()).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_dn.png") 
                Image.fromarray(pred255_dn.squeeze()).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_exp.png") 
                Image.fromarray(pred255_exp.squeeze()).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_mid.png") 
                Image.fromarray(pred255_mid.squeeze()).convert(color_mode).save(save_path)
            else:
                color_mode = 'L'
                save_path = os.path.join(save_dir, f"{name}_clean.png")
                Image.fromarray(origin255).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_noisy.png")    
                Image.fromarray(noisy255).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_dn.png") 
                Image.fromarray(pred255_dn).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}_exp.png") 
                Image.fromarray(pred255_exp).convert(color_mode).save(save_path)

                save_path = os.path.join(save_dir, f"{name}-_mid.png") 
                Image.fromarray(pred255_mid).convert(color_mode).save(save_path)



LearningRate of Epoch 1 = 0.0001


100%|██████████| 229/229 [02:26<00:00,  1.56it/s]


LearningRate of Epoch 2 = 0.0001


100%|██████████| 229/229 [02:26<00:00,  1.57it/s]


LearningRate of Epoch 3 = 0.0001


...

100%|██████████| 229/229 [02:26<00:00,  1.57it/s]


LearningRate of Epoch 99 = 6.25e-06


100%|██████████| 229/229 [02:26<00:00,  1.56it/s]


LearningRate of Epoch 100 = 6.25e-06


100%|██████████| 229/229 [02:26<00:00,  1.57it/s]


In [ ]:
!ls data/test/small_b2u_g209

Tile_001-004-000_0-000.png  Tile_001-030-000_0-000.png


In [43]:
img = cv2.imread('data/test/small_b2u_g209/Tile_001-008-000_0-000.png',0)
crop = img[1000:1480, 1000:1640]
cv2.imwrite ('data/test/small_b2u_g209/Tile_001-008-000_0-000_1.png',crop )

True

In [34]:
torch.cuda.empty_cache()